<a href="https://colab.research.google.com/github/anshudaur/TensorFlowProjects/blob/master/RNN_LSTM_GRU_languageModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# %tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import pandas as pd
import copy
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [0]:
!python3 prepare_data2.py the-king-james-bible.txt shake \\n\\n+ -m 500

2020-06-02 09:10:43.568354: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 24607 sequences...
Longest sequence is 2552 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
24057 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
24057 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [0]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("shake.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
# data = data.map(lambda x: parse_seq(x, 200))

# Now no fixed length sequence
data = data.map(parse_seq)
# data = data.padded_batch(batch_size, (-1,), drop_remainder=True)


# a map from characters to indices
vocab = pickle.load(open("shake_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print('Vocab : {}\n'.format(vocab))
print('Vocab Size : {}\n'.format(vocab_size))

Vocab : {'I': 3, '9': 4, 'w': 5, 'n': 6, 'K': 7, 'b': 8, '8': 9, '.': 10, 'o': 11, '1': 12, '6': 13, ' ': 14, '0': 15, '7': 16, 'N': 17, 'O': 18, 'Y': 19, '\n': 20, 'L': 21, 'A': 22, 'C': 23, 'V': 24, 'P': 25, 'x': 26, 'W': 27, 'G': 28, 'd': 29, 'r': 30, 'u': 31, 't': 32, 'p': 33, '4': 34, 'z': 35, 'f': 36, 'E': 37, 'j': 38, 'i': 39, 'Z': 40, '2': 41, '5': 42, '?': 43, 'S': 44, 'g': 45, 'c': 46, 'U': 47, 'J': 48, '*': 49, 'M': 50, 'T': 51, '!': 52, 'h': 53, ')': 54, 'l': 55, ';': 56, 'B': 57, 'F': 58, 'Q': 59, 'm': 60, 'a': 61, 'R': 62, 's': 63, 'q': 64, '(': 65, '-': 66, ',': 67, ':': 68, 'v': 69, "'": 70, '\ufeff': 71, 'k': 72, 'D': 73, '3': 74, 'y': 75, 'e': 76, 'H': 77, '<PAD>': 0, '<S>': 1, '</S>': 2}

Vocab Size : 78



In [0]:
# lambda function to convert data to one hot encoding format for each character
# using Lambda function of keras to add the one_hot to sequential model
one_hot_encode = tf.keras.layers.Lambda(lambda data: tf.one_hot(indices=data,depth=vocab_size));

In [0]:
def lstm_model():
    # lstm sequential model
    model = tf.keras.Sequential()
    # The LSTM input layer should be 3D. The dimensions are : samples, time steps, and features.
    model.add(tf.keras.layers.LSTM(512, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
        kernel_initializer='glorot_uniform', return_sequences=True, stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    # Provide the inputs to the LSTM layer.(samples=batch_size, time steps=None, features=vocab_size)
    model.build([batch_size , None , vocab_size])
    model.summary()
    return model

In [0]:
def gru_model():
    # lstm sequential model
    model = tf.keras.Sequential()
    # The LSTM input layer should be 3D. The dimensions are : samples, time steps, and features.
    model.add(tf.keras.layers.GRU(512, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
        kernel_initializer='glorot_uniform', return_sequences=True, stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    model.build([batch_size , None , vocab_size])
    model.summary()
    return model

In [0]:
def simple_RNN_model():
    # lstm sequential model
    model = tf.keras.Sequential()
    # The LSTM input layer should be 3D. The dimensions are : samples, time steps, and features.
    model.add(tf.keras.layers.SimpleRNN(512, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal', bias_initializer='zeros', return_sequences=True, stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    model.build([batch_size , None , vocab_size])
    model.summary()
    return model

In [0]:
# sequence_length = 200 #already given
#each batch should be batch_size x seq_len x vocab_size
batch_size=128

test_data=data
# test_data = test_data.batch(batch_size,drop_remainder=True)
test_data = test_data.padded_batch(batch_size,(-1,),drop_remainder=True)
print("Test data : ", test_data)

# train_data = data.map(one_hot_encode)
train_data=data
# train_data = data.map(one_hot_encode)
train_data = train_data.padded_batch(batch_size,(-1,),drop_remainder=True)

# train_data = train_data.batch(batch_size,drop_remainder=True)
print("Train data : ",train_data)

#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()

Test data :  <PaddedBatchDataset shapes: (128, None), types: tf.int32>
Train data :  <PaddedBatchDataset shapes: (128, None), types: tf.int32>


In [0]:
def train_model(rnn_model, train_data, test_data, epochs):
    model = rnn_model
    model_loss = 0
    for epoch in range(epochs):    
        for batch_num,(x, y) in enumerate(zip(train_data,test_data)):
            x_train = x[:,:-1]
            y_label = y[:,1:]
            # counting actual elements as the padded values are zero.
            nonzero_elements = tf.math.count_nonzero(x, 1)
            # Subtracting 1 as the last elememt is not used as input 
            nonzero_elements -= 1
            # 2D mask batch x time
            masking = tf.sequence_mask(nonzero_elements, dtype=tf.float32)
            # print('x_train : {}\n'.format(x_train))
            with tf.GradientTape() as tape:
                # one hot encoding for input sequence characters
                x_train = model(one_hot_encode(x_train))
                # (batch x time) loss and mask multiplication with softmax_cross_entropy
                loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=x_train, labels=y_label)
                loss = tf.reduce_sum(loss * masking)
                # cast to float32 from int64
                real_elements = tf.cast(nonzero_elements, tf.float32)
                # using reduced_sum instead of reduced_mean
                loss /= tf.reduce_sum(real_elements)
            if not batch_num % 100:  
                print("Epoch ::",epoch, " loss :: ",loss)  
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
#             metric(y_label, x_train) 

        # Display metrics at the end of each epoch.
#         accuracy = metric.result()    
#         print('Training acc over epoch' , epoch,' : ',(float(accuracy)))
        # Reset training metrics at the end of each epoch
#         metric.reset_states()
        model.reset_states()
        

In [0]:
# model variables : recommended 512 hidden units (batch size 128 and Adam optimizer, 20 or so )
train_model(lstm_model(),train_data,test_data,epochs=20)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  1210368   
_________________________________________________________________
dense (Dense)                multiple                  40014     
Total params: 1,250,382
Trainable params: 1,250,382
Non-trainable params: 0
_________________________________________________________________
Epoch :: 0  loss ::  tf.Tensor(4.362029, shape=(), dtype=float32)
Epoch :: 0  loss ::  tf.Tensor(2.3848474, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.0934634, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.0286229, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(1.9367489, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(1.9310391, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(1.8654876, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(1.8709047

In [0]:
# model variables : recommended 512 hidden units (batch size 128 and Adam optimizer, 20 or so )
train_model(simple_RNN_model(),train_data,test_data,epochs=20)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       multiple                  302592    
_________________________________________________________________
dense_2 (Dense)              multiple                  40014     
Total params: 342,606
Trainable params: 342,606
Non-trainable params: 0
_________________________________________________________________
Epoch :: 0  loss ::  tf.Tensor(4.382275, shape=(), dtype=float32)
Epoch :: 0  loss ::  tf.Tensor(2.8626513, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.4816494, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.3797395, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(2.2638786, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(2.2185423, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(2.1526468, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(2.1383965, 

In [0]:
# model variables : recommended 512 hidden units (batch size 128 and Adam optimizer, 20 or so )
train_model(gru_model(),train_data,test_data,epochs=20)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    multiple                  909312    
_________________________________________________________________
dense_1 (Dense)              multiple                  40014     
Total params: 949,326
Trainable params: 949,326
Non-trainable params: 0
_________________________________________________________________
Epoch :: 0  loss ::  tf.Tensor(4.360831, shape=(), dtype=float32)
Epoch :: 0  loss ::  tf.Tensor(2.5551348, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.236947, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(2.1534698, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(2.0711107, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(2.0442955, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(1.9975369, shape=(), dtype=float32)
Epoch :: 3  loss ::  tf.Tensor(1.9808009, s

Repeating GRU layers to see if performance improves

In [0]:
def gru_model_2():
    # lstm sequential model
    model = tf.keras.Sequential()
    # The LSTM input layer should be 3D. The dimensions are : samples, time steps, and features.
    model.add(tf.keras.layers.GRU(512, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
        kernel_initializer='glorot_uniform', return_sequences=True, stateful=True))
    model.add(tf.keras.layers.GRU(512, activation='tanh', recurrent_activation='sigmoid', use_bias=True,
        kernel_initializer='glorot_uniform', return_sequences=True, stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    model.build([batch_size , None , vocab_size])
    model.summary()
    return model

In [0]:
# model variables : recommended 512 hidden units (batch size 128 and Adam optimizer, 20 or so )
train_model(gru_model_2(),train_data,test_data,epochs=30)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  multiple                  909312    
_________________________________________________________________
gru_3 (GRU)                  multiple                  1575936   
_________________________________________________________________
dense_1 (Dense)              multiple                  40014     
Total params: 2,525,262
Trainable params: 2,525,262
Non-trainable params: 0
_________________________________________________________________
Epoch :: 0  loss ::  tf.Tensor(4.3540874, shape=(), dtype=float32)
Epoch :: 0  loss ::  tf.Tensor(3.3043652, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(6.034909, shape=(), dtype=float32)
Epoch :: 1  loss ::  tf.Tensor(3.289881, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(3.1638744, shape=(), dtype=float32)
Epoch :: 2  loss ::  tf.Tensor(3.0776439,

Unable to make the text generation work

In [0]:
def sample(n_seq):        
  for _ in range(n_seq):
    model = gru_model_2()
    char = 1      
    txt = []
    gen = tf.Variable([[tf.one_hot(char, vocab_size)]])
    x_train = gen[:,:-1]
    out = model(one_hot_encode(x_train)) #last executed GRU model
    probs = tf.reshape(out),[-1]).numpy()
    txt.append(np.argmax(probs))
    #print(txt)
      

    print("".join([ind_to_ch[ind] for ind in txt]).replace("</S>", "\n\n"))
        
sample(100)


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_34 (GRU)                 multiple                  909312    
_________________________________________________________________
gru_35 (GRU)                 multiple                  1575936   
_________________________________________________________________
dense_17 (Dense)             multiple                  40014     
Total params: 2,525,262
Trainable params: 2,525,262
Non-trainable params: 0
_________________________________________________________________


NotFoundError: ignored